# Back Propagation
> 误差反向传播

A brilliant idea: calculate the influence of the inputs to this node, propagate the influence back to the previous node based on the *chain law*.

Finally get the influence, aka the gradient of every node to the loss function.

This method will be significantly faster than just calculating every gradient.

## Implementation of Basic Layers
### Implement Layer of Multiply
and use the class to implement this picture:
<div align="center"><img src="example1.png" width=50% height=50%><div\>

In [72]:
class mulLayer:
    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        return out
    
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        return dx, dy
    
    # according to z = xy: dz/dx = y; dz/dy = x
appleCnt = 2
pricePerApple = 100
taxRatio = 1.1

applePriceLayer = mulLayer()
taxPriceLayer = mulLayer()

applePrice = applePriceLayer.forward(pricePerApple, appleCnt)
taxPrice = taxPriceLayer.forward(applePrice, taxRatio)

print(f"when tax ratio is {taxRatio}, an apple cost {pricePerApple}, buying {appleCnt} apple(s) costs {taxPrice}")

when tax ratio is 1.1, an apple cost 100, buying 2 apple(s) costs 220.00000000000003


then I will use the backward method to show the influence of the apple count and tax ratio to the final price

In [73]:
dPrice = 1
dApplePrice, dTaxRatio = taxPriceLayer.backward(dPrice)
dPricePerApple, dAppleCnt = applePriceLayer.backward(dApplePrice)

print(f"influence of the apple cnt is {dAppleCnt}")
print(f"influence of the price per apple is {dPricePerApple}")
print(f"influence of the tax ratio is {dTaxRatio}")

influence of the apple cnt is 110.00000000000001
influence of the price per apple is 2.2
influence of the tax ratio is 200


### Implement Layer of Add
and use the method to implement this picture:

<div align="center"><img src="example2.png" width=50% height=50%><div\>

In [74]:
class addLayer:
    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout
        dy = dout
        return dx, dy
    
    # according to z = x + y: dz/dx = 1; dz/dy = 1

In [75]:
# forward
appleCnt = 2
pricePerApple = 100
orangeCnt = 3
pricePerOrange = 150
taxRatio = 1.1

applePriceLayer = mulLayer()
orangePriceLayer = mulLayer()
totalPriceLayer = addLayer()
taxLayer = mulLayer()

applePrice = applePriceLayer.forward(appleCnt, pricePerApple)
orangePrice = orangePriceLayer.forward(orangeCnt, pricePerOrange)
totalPrice = totalPriceLayer.forward(applePrice, orangePrice)
finalPrice = taxLayer.forward(totalPrice, taxRatio)

print(f"an apple cost {pricePerApple}")
print(f"an orange cost {pricePerOrange}")
print(f"when tax ratio is {taxRatio}")
print(f"buying {appleCnt} apple(s) and {orangeCnt} orange(s) costs {finalPrice}")

an apple cost 100
an orange cost 150
when tax ratio is 1.1
buying 2 apple(s) and 3 orange(s) costs 715.0000000000001


In [76]:
# backward
dPrice = 1

dTotalPrice, dTaxRatio = taxLayer.backward(dPrice)
dApplePrice, dOrangePrice = totalPriceLayer.backward(dTotalPrice)
dAppleCnt, dPricePerApple = applePriceLayer.backward(dApplePrice)
dOrangeCnt, dPricePerOrange = orangePriceLayer.backward(dOrangePrice)

print(f"influence of the apple cnt is {dAppleCnt}")
print(f"influence of the price per apple is {dPricePerApple}")
print(f"influence of the orange cnt is {dOrangeCnt}")
print(f"influence of the price per orange is {dPricePerOrange}")
print(f"influence of the tax ratio is {dTaxRatio}")

influence of the apple cnt is 110.00000000000001
influence of the price per apple is 2.2
influence of the orange cnt is 165.0
influence of the price per orange is 3.3000000000000003
influence of the tax ratio is 650


## Implement Layers of Activation Functions
### ReLU
$$
y = \begin{cases}
x &(x> 0)\\
0 &(x\le 0)
\end{cases}
$$

According to the definition of the function:
$$
\frac{\partial y}{\partial x} = 
\begin{cases}
1 &(x> 0)\\
0 &(x\le 0)
\end{cases}
$$

In [77]:
# ReLU
class ReLU:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout.copy()

        return dx

### Sigmoid
$$
y = \frac{1}{1 + e^{-x}}
$$

Calculate picture as below:
<div align="center"><img src="example3.png" width=50% height=50%></div>

It seems a little big complex, try to implement it.

In [78]:
import numpy as np

class expLayer:
    def __init__(self):
        self.x = None

    def forward(self, x):
        self.x = x
        out = np.exp(x)

        return out
    
    def backward(self, dout):
        dx = dout * np.exp(self.x)

        return dx
    
class oneDivLayer:
    def __init__(self):
        self.x = None

    def forward(self, x):
        self.x = x
        out = 1 / x

        return out
    
    def backward(self, dout):
        dx = dout * (-(1 / (self.x * self.x)))

        return dx
    
class sigmoidLayer:
    def __init__(self):
        self.firstLayer = mulLayer()
        self.secondLayer = expLayer()
        self.thirdLayer = addLayer()
        self.forthLayer = oneDivLayer()

    def forward(self, x):
        firstOutput = self.firstLayer.forward(x, -1)
        secondOutput = self.secondLayer.forward(firstOutput)
        thirdOutput = self.thirdLayer.forward(secondOutput, 1)
        forthOutput = self.forthLayer.forward(thirdOutput)

        return forthOutput
    
    def backward(self, dout):
        firstBack = self.forthLayer.backward(dout)
        secondBack, temp = self.thirdLayer.backward(firstBack)
        thirdBack = self.secondLayer.backward(secondBack)
        forthBack, temp = self.firstLayer.backward(thirdBack)

        return forthBack
    
# test
test = sigmoidLayer()
output = test.forward(np.array([1, 1, 1]))
print(output)

backwardOut = test.backward(np.array([1, 1, 1]))
print(backwardOut)

[0.73105858 0.73105858 0.73105858]
[0.19661193 0.19661193 0.19661193]


However, the method above is too slow and complex.

We should use math to help: (put input x, output y for sigmoid)
$$
\frac{\partial y}{\partial x} = Ey^2e^{-x} = Ey(1 - y)
$$

In [79]:
class sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        
        return out
    
    def backward(self, dout):
        dx = dout * self.out * (1 - self.out)
        return dx

# test
test = sigmoidLayer()
output = test.forward(np.array([1, 1, 1]))
print(output)

backwardOut = test.backward(np.array([1, 1, 1]))
print(backwardOut)

[0.73105858 0.73105858 0.73105858]
[0.19661193 0.19661193 0.19661193]


## Affine & Softmax Layer
### Affine
Affine is the layer that do the equation:
$$
Y = X\cdot W + B
$$

as the picture below:
<div align="center"><img src="example4.png" width=50% height=50%></div>

To implement the layer, use the conclusion below:
$$
\begin{align*}
&\frac{\partial L}{\partial X} = \frac{\partial L}{\partial Y} \cdot W^T\\
&\frac{\partial L}{\partial W} = X^T\cdot \frac{\partial L}{\partial Y}
\end{align*}
$$

In the batch version, there will be $N$ $X$ and $N$ $W$, and one $B$

We can use the same equation for the BP, just see the shape:
```
x.shape = n * a
w.shape = a * b
b.shape = b
y.shape = n * b

dl/dy shape = n * b
dl/dx shape = n * b - b * a = n * a
dl/dw shape = a * n - n * b = a * b
dl/db, use sum
```

In [ ]:
# affine layer
class affine:
    def __init__(self, w, b):
        self.w = w
        self.b = b
        self.x = None
        self.dw = None
        self.db = None

    def forward(self, x):
        self.x = x
        result = np.dot(x, self.w) + self.b
        return result
    
    def backward(self, dl):
        dx = np.dot(dl, self.dw.T)
        self.dw = np.dot(self.x.T, dl) 
        self.db = np.sum(dl, axis = 0)

        return dx